In [ ]:
# our own script
from DreadScraper import DreadSession

# visual
from tqdm.auto import tqdm

# data handeling
import json
import csv
import pandas as pd

# not used yet
import networkx as nx
from networkx.readwrite import json_graph
from joblib import Parallel, delayed
import itertools
import os
from time import sleep
import random
import community
import networkit as nk
import matplotlib.pyplot as plt
import numpy as np


# Motivation

- What is your dataset?
- Why did you choose this/these particular dataset(s)?
- What was your goal for the end user's experience?

The dataset used in this project is sourced from <span style="color: purple; font-weight: bold;">Dread</span>, specifically the <span style="color: purple; font-weight: bold;">/d/DarkNetMarkets</span> forum.


This dataset provides a rich foundation for analyzing community behavior, market activity, and communication trends within darknet marketplaces.

It also seemed interesting to analyze the linked onion addresses from the content of the posts and comments.










mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst mere tekst

---

# Basic Stats

- Write about your choices in data cleaning and preprocessing
- Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)

The dataset consists of:

<div style="border-left: 4px solid #2ca02c; padding-left: 12px; line-height: 1.6em;">
<pre style="font-size: 14px; line-height: 1.6em;">
<span style="color: #2ca02c; font-weight: bold;">30,793 users</span>        — The number of users who have written something (out of 155,000+ users)
<span style="color: #2ca02c; font-weight: bold;">64.3 MB</span>             — Total file size
<span style="color: #2ca02c; font-weight: bold;">6 variables</span>         — Describing user activity and content
<span style="color: #2ca02c; font-weight: bold;">16,116 posts</span>        — The total number of posts
<span style="color: #2ca02c; font-weight: bold;">181,400 comments</span>    — Total number of comments on every post and comment
<span style="color: #2ca02c; font-weight: bold;">197,516 entries</span>     — The total number rows in the dataset
<span style="color: #2ca02c; font-weight: bold;">19.4.2025</span>           — Start date for data collection
<span style="color: #2ca02c; font-weight: bold;">21.4.2025</span>           — End date for data collection
</pre>
</div>

Each row in the dataset corresponds to either a post or a comment, and contains the following variables:

<div style="border-left: 4px solid #9467bd; padding-left: 12px; line-height: 1.6em;">
<pre style="font-size: 14px; line-height: 1.6em;">
<span style="color: #9467bd; font-weight: bold;">type</span>                — Entry type; <span style="color: #ff7f0e;">post</span> or <span style="color: #ff7f0e;">comment</span>
<span style="color: #9467bd; font-weight: bold;">id</span>                  — Unique identifier
<span style="color: #9467bd; font-weight: bold;">parent_id</span>           — Parent reference; <span style="color: #ff7f0e;">None</span> if it is a post, otherwise the id of the post or comment it replies to
<span style="color: #9467bd; font-weight: bold;">title</span>               — Post title (blank for comments)
<span style="color: #9467bd; font-weight: bold;">author</span>              — Author username, which is also the unique identifier of the user
<span style="color: #9467bd; font-weight: bold;">content</span>             — Post or comment text
</pre>
</div>


Initially 1474 links were found in the 197,516 texts. After removing v1 & v2 links and duplicates 212 addresses remained, of whitch only 59 were responsive. Version 1 and 2 links are recognized by their hash lenght. See the text below for an explanation on onion addresses.

<div style="border-left: 4px solid #2ca02c; padding-left: 12px; line-height: 1.6em;">
<pre style="font-size: 14px; line-height: 1.6em;">
<span style="color: #2ca02c; font-weight: bold;">59 addresses</span>        — Responsive v3 onion addresses
<span style="color: #2ca02c; font-weight: bold;">135.4 KB</span>            — Total file size
<span style="color: #2ca02c; font-weight: bold;">3 variables</span>         — Describing the address and its content
<span style="color: #2ca02c; font-weight: bold;">30 unique</span>           — Meaning that 29 of them are additional addresses to identical sites
<span style="color: #2ca02c; font-weight: bold;">2.5.2025</span>            — Date of data collection
</pre>
</div>

The dataset contains the following variables:

<div style="border-left: 4px solid #9467bd; padding-left: 12px; line-height: 1.6em;">
<pre style="font-size: 14px; line-height: 1.6em;">
<span style="color: #9467bd; font-weight: bold;">address</span>             — The unique onion address
<span style="color: #9467bd; font-weight: bold;">alias</span>               — Page name/alias of the onion address, either automatic from HTML <span style="color: green;"><</span><span style="color: blue;">title</span><span style="color: green;">></span> or manually added
<span style="color: #9467bd; font-weight: bold;">content</span>             — The entire HTML <span style="color: green;"><</span><span style="color: blue;">body</span><span style="color: green;">></span> text content
</pre>
</div>

---

<span style="color: #1f77b4; font-weight: bold;">Onion addresses</span> are unique identifiers used to access hidden services on the Tor network. Unlike traditional domains that rely on DNS and IP addresses, onion addresses are resolved through Tor’s <span style="color: #1f77b4; font-weight: bold;">distributed hash table (DHT)</span>, allowing users to connect to the endpoints of servers. This endpoint is similar to the endpoint of your onion circuit. This is what keeps you both anonymous. If you dont know about onion routing, imagine connecting to 3 VPNs in a row. the last "VPN" is the endpoint.

Each address is a hash derived from a cryptographic public key, serving as both the server’s <span style="color: #1f77b4; font-weight: bold;">identity</span> and its <span style="color: #1f77b4; font-weight: bold;">encryption mechanism</span>. This eliminates the need for HTTPS or certificate authorities — the Tor protocol handles end-to-end encryption internally.

Earlier versions used <span style="color: #1f77b4; font-weight: bold;">16-character</span> addresses (<span style="color: #1f77b4; font-weight: bold;">v2</span>), but since <span style="color: #1f77b4; font-weight: bold;">October 2021</span>, only <span style="color: #1f77b4; font-weight: bold;">v3</span> addresses are valid, which are <span style="color: #1f77b4; font-weight: bold;">56 character</span> long and cryptographically stronger.

<span style="font-weight: bold;">Regex pattern for v3 onion addresses:</span> <span style="color: #666666;">/</span><span style="color: #2ca02c;">(</span><span style="color: #1f77b4;">http</span><span style="color: purple;">://</span><span style="color: #ff7f0e;">[a-z2-7]{56}</span><span style="color: yellow;">\.onion</span><span style="color: #2ca02c;">)</span><span style="color: #666666;">/</span>

<span style="font-weight: bold;">The Dread forum address: </span> <span style="color: #1f77b4;">http</span><span style="color: purple;">://</span><span style="color: #ff7f0e;">dreadytofatroptsdj6io7l3xptbet6onoyno2yv7jicoxknyazubrad</span><span style="color: yellow;">\.onion</span>

---


There were quite a bit of hurdles to overcome, when scraping Dread. Forums and pages with illegal content can't enforce legal TOS, so they have more agressive anti-scraping than clear web sites. When the scraper was detected a significant throtteling was applied to the requests, and we had to wait for several hours befor we could scrape at full speed again. This is one of the primary why the data was collected over three days. We have made a tool - <span style="color: orange;">DreadScraper.DreadSession</span> - to use different kinds of obfuscation techniques.

If you are interested in the full code for the webscraper, please check out the <span style="color: #1f77b4; font-weight: bold;">Gitlab repository</span> for the project: <span style="color: #1f77b4;">https://gitlab.gbar.dtu.dk/s224234/socsciproject</span>

In [2]:
session = DreadSession()
'''
We need a CAPTCHA token and a session token to scrape the Dread forums.
This is one of dreads many anti-bot measures.
you can get the cookies by going to the page manually and copying them from your browser.

The cookies below are not going to work for you, they expire after a few hours.
'''
cookies = {
            "dcap": "/zi8jE705mfzXrpRh0boFJKWc7akvJhWD3pHOSwmbyaokQtdYCoIr7lB8MgLqclV/zZI0UsSChL4cjHYfU5J2hyfPzsS94nytelAL12OZcxXM2hhmXkEE8dx9AGZNoGEZzkVTyAfoWiAJxdfAT299GEt8nggjm80wfzxYEZLwlkitFFcbiI=",
            "dread": "6mp2h4rrrhjq3a8k9af7m7pubg"
        }
session.cookies = cookies

# or initialize the session with the cookies
session = DreadSession(cookies=cookies)

# you can check your access to onion service and dread forums
# by running the following command
tor,dread = session.status(verbose=True)

if not tor:
    print("you dont have a working tor connection, or you dont have internet access")
if not dread:
    print("your cookies are probably wrong")


Checking Tor configuration...[OK] Tor is configured correctly.

Checking Dread access...[OK] Connected to Dread successfully.


--- 

The data collection was done using the <span style="color: #1f77b4; font-weight: bold;">BeautifulSoup</span> library for parsing html.

The structure of the forum is similar to that of Reddit, with posts and comments. The posts are displayed in a list format. The pagination process is quite simple and can be done by simply adding page numbers and sorting method to the url.

> <span style="color: #888888;">http://</span><span style="color: #888888;">dreadytofatroptsdj6io7l3xptbet6onoyno2yv7jicoxknyazubrad.onion</span><span style="color: purple;">/d/darknetmarkets/</span><span style="color: red;">?</span><span style="color:rgb(81, 182, 255);">sort</span><span style="color: red;">=</span><span style="color: #2ca02c;">new</span><span style="color: red;">&</span><span style="color:rgb(81, 182, 255);">p</span><span style="color: red;">=</span><span style="color: #2ca02c;">1</span>


and the html structure allows us to easily extract the post links

```html
<div class="postBoard">
    <div class="item">
        <div class="postTop">
            <a class="title" href="/post/ID_HERE">Title of post</a>
            ...
        </div>
        <div class="postMain"> ... </div>
    </div>
    <div class="item"> ... </div>
    <div class="item"> ... </div>
    <div class="item"> ... </div>
    <div class="item"> ... </div>
</div>
```

In [3]:
# lets just get the 20 ids on the first page of DarkNetMarkets
subforum = 'darknetmarkets'
post_ids = session.get_post_ids(subforum, start_from=1, sort="new", pagination=False, Verbrose=True) # getting ids from the first page

print(post_ids)

Posts found: 20
['9c2bcd429963bf49c407', '8f3fb59719ad04288227', '5def8253e4e88845a0db', 'e6a96d14edc2728999fa', 'c5d49ea61ab450006bf6', '5cedae0638227dec20f9', 'ba8587094e96c2874ccf', 'bb60ac95f16bac956c88', 'cebb8bbd054a6ebe3e46', 'b8934de68253f2873500', 'd303bfb780e2d8aef678', '7ac22dbde8d6cce6b198', 'f974853c23374762a7e6', '5cfd9927c6b8101ed443', 'd2b177a18ef0d3fddf43', '13d60ab23ec6acc6bc4f', '2a250b103872c0b8a3bb', 'dee9585e316b7c53e19b', '8dbb8627421d205c52f4', '8a85394e70690bc915ca']


---

When we click on a post, we are taken to a new page with the post content and the comments. The url for the post is simply the id of the post, which is also the id of the post in the dataset. The url looks like this:

> <span style="color: #888888;">http://</span><span style="color: #888888;">dreadytofatroptsdj6io7l3xptbet6onoyno2yv7jicoxknyazubrad.onion</span><span style="color: purple;">/post/ID_HERE</span>

The structure of the html on the posts are convenient to handle for our purpose. The structure we are interested in, can be summed down to the following:

```html
<div class="postBoard viewPost">
    <div class="item">
        <div class="postTop">
            <a class="title" href="/post/ID_HERE">Title of post</a>
            <div class="author"> by <a href="/u/username">/u/username</a>...</div>
            ...
        </div>
        <div class="postMain">
            <div class="postContent"> text content </div>
            ...
        </div>
    </div>
</div>

...

<div class="postComments">
    <div class="comment">

        <div class="commentContent">
            <div class="top"><a href="/u/username" class="username">/u/username</a>...</div>
            <div class="commentBody"> text content </div>
            ...
        </div>
        <div class="comment">
            <div class="comment"> ... </div>
            <div class="comment"> ... </div>
            ...
        </div>
        ...
    </div>
    <div class="comment"> ... </div>
    <div class="comment"> ... </div>
    <div class="comment"> ... </div>
</div>
```

In [5]:
output_csv_path = "Data/explainer_posts.csv"

# Check if we need to write headers for new file
file_exists = os.path.exists(output_csv_path)

with open(output_csv_path, "a", encoding="utf-8", newline="") as csvfile:
    writer = None

    for post_id in tqdm(post_ids, desc="Fetching post content", unit="post"):
        _, data = session.get_post_content(post_id)

        if not data:
            continue  # skip empty results

        if writer is None:
            # Initialize CSV writer with fieldnames from first dict
            writer = csv.DictWriter(csvfile, fieldnames=data[0].keys())
            if not file_exists:
                writer.writeheader()

        # Write each comment row
        for row in data:
            writer.writerow(row)

Fetching post content:   0%|          | 0/20 [00:00<?, ?post/s]

In [8]:
data = pd.read_csv(output_csv_path, encoding="utf-8")
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   type       355 non-null    object
 1   id         355 non-null    object
 2   parent_id  307 non-null    object
 3   title      48 non-null     object
 4   author     355 non-null    object
 5   content    355 non-null    object
dtypes: object(6)
memory usage: 16.8+ KB


,type,id,parent_id,title,author,content
0,post,9c2bcd429963bf49c407,NaN,where do I find pornography of small boy fucki...,NewGroundDemon,I have tried finding it but nothing is working...
1,post,8f3fb59719ad04288227,NaN,"ƤȃŷƤȃl Ḟȃsṯ Țrȃńsfer Ḋōing , Ǵeṯ in ṯōuch",FastCleanPayPalTranaFer4m,"You can talk with us whenever on ,,,...T..,,,...."
2,post,5def8253e4e88845a0db,NaN,truegenerics Legit??,NierAuto,Wanna buy some accutane from him on Dark Matte...
3,post,e6a96d14edc2728999fa,NaN,Dark Matter is using two naked lesbians in Dre...,GhostShelf,"Is it just me, or is Dark Matter Market using ..."
4,comment,c-609e781b276644d38b,c-f25ce477b1e85ca723,NaN,DrugHub,"That's not my line, Paris said it a while ago ..."
5,comment,c-f25ce477b1e85ca723,c-a4b9370c4b7264c5a0,NaN,GhostShelf,Short and true. Best one so far LMAO
6,comment,c-a4b9370c4b7264c5a0,e6a96d14edc2728999fa,NaN,DrugHub,Tits get clicks.
7,comment,c-6657bdc06472e76d72,c-b4bd1aacb43322698b,NaN,GhostShelf,Same here. These guys know marketing — no doub...
8,comment,c-b4bd1aacb43322698b,e6a96d14edc2728999fa,NaN,Wilhelmo,Yes they do and it caught my attention as well...
9,comment,c-535c3e26a7dfd68638,c-a993cb327a04d11c61,NaN,Phobos36,"/u/Fyodor-MD \r Do you not like naked wymenz, ..."


# Tools, Theory and Analysis

- Talk about how you've worked with text, including regular expressions, unicode, etc.





- Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.






- How did you use the tools to understand your dataset?





# Discussion

- What went well?





- What is still missing? What could be improved? Why?




